In [1]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

from sklearn.model_selection import train_test_split

In [2]:
label_name = "math"

dataset = pd.read_csv(f"../dataset/{label_name}_dataset.csv")
dataset = pd.get_dummies(dataset)

train, test = train_test_split(dataset, test_size=0.5, random_state=42, shuffle=True)

dataset.head()

,branch-misses_FEATURE_CONFIG,branch-misses_INTERCEPT,branch-misses_R-VAL,branches_FEATURE_CONFIG,branches_INTERCEPT,branches_R-VAL,context-switches_FEATURE_CONFIG,context-switches_INTERCEPT,context-switches_R-VAL,cpu-migrations_FEATURE_CONFIG,...,stalled-cycles-frontend_FEATURE_TYPE_LOGLOG_POLYNOMIAL,stalled-cycles-frontend_FEATURE_TYPE_LOG_POLYNOMIAL,stalled-cycles-frontend_FEATURE_TYPE_POLYNOMIAL,stalled-cycles-frontend_FEATURE_TYPE_POWER,task-clock_FEATURE_TYPE_FACTORIAL,task-clock_FEATURE_TYPE_FRACTIONAL_POWER,task-clock_FEATURE_TYPE_LOGLOG_POLYNOMIAL,task-clock_FEATURE_TYPE_LOG_POLYNOMIAL,task-clock_FEATURE_TYPE_POLYNOMIAL,task-clock_FEATURE_TYPE_POWER
0,0.0,12176.278640,5.274472,1.0,354489.841881,31.023435,0,0.0,0.0,0,...,0,1,0,0,0,0,0,0,1,0
1,0.8,12300.866448,140.866882,0.1,360567.282593,278.416553,0,0.0,0.0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.0,12396.648839,38.928139,1.0,358483.822954,81.019933,0,0.0,0.0,0,...,0,0,1,0,0,0,0,0,1,0
3,4.0,12358.582719,-0.000011,1.0,355754.502857,256.760672,0,0.0,0.0,0,...,0,0,0,1,0,1,0,0,0,0
4,1.0,12313.598463,3.715744,1.0,354852.459189,327.289900,0,0.0,0.0,0,...,0,1,0,0,0,1,0,0,0,0


In [3]:
dataset_features = train.copy()
dataset_labels = train.copy().pop('label')

In [4]:
dataset_features = np.array(dataset_features)

In [5]:
normalize = preprocessing.Normalization()
normalize.adapt(dataset_features)

model = tf.keras.Sequential([
  normalize,
  layers.Dense(1024),
  layers.Dense(512),
  layers.Dense(512),
  layers.Dense(128),
  layers.Dense(128),
  layers.Dense(64),
  layers.Dense(64),
  layers.Dense(32),
  layers.Dense(32),
  layers.Dense(8),
  layers.Dense(8),  
  layers.Dense(4),
  layers.Dense(2),
  layers.Dense(1, activation='sigmoid')
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer = tf.optimizers.Adam(),
              metrics=['accuracy'])

2021-10-09 14:29:31.039936: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-09 14:29:31.101029: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [12]:
model.fit(dataset_features, dataset_labels, epochs=1000)

Epoch 1/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.4045e-06 - accuracy: 1.0000
Epoch 2/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.2196e-06 - accuracy: 1.0000
Epoch 3/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.0667e-06 - accuracy: 1.0000
Epoch 4/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.4327e-07 - accuracy: 1.0000
Epoch 5/1000
90/90 [==============================] - 1s 8ms/step - loss: 8.3874e-07 - accuracy: 1.0000
Epoch 6/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.5091e-07 - accuracy: 1.0000
Epoch 7/1000
90/90 [==============================] - 1s 7ms/step - loss: 6.7551e-07 - accuracy: 1.0000
Epoch 8/1000
90/90 [==============================] - 1s 7ms/step - loss: 6.0987e-07 - accuracy: 1.0000
Epoch 9/1000
90/90 [==============================] - 1s 7ms/step - loss: 5.5303e-07 - accuracy: 1.0000
Epoch 10/1000
90/90 [==============================] - 1s 6ms/st

90/90 [==============================] - 1s 6ms/step - loss: 8.4987e-09 - accuracy: 1.0000
Epoch 80/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.0942e-09 - accuracy: 1.0000
Epoch 81/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.7079e-09 - accuracy: 1.0000
Epoch 82/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.3379e-09 - accuracy: 1.0000
Epoch 83/1000
90/90 [==============================] - 1s 6ms/step - loss: 6.9850e-09 - accuracy: 1.0000
Epoch 84/1000
90/90 [==============================] - 1s 6ms/step - loss: 6.6503e-09 - accuracy: 1.0000: 0s - loss: 6.6206e-09 - accuracy
Epoch 85/1000
90/90 [==============================] - 1s 6ms/step - loss: 6.3354e-09 - accuracy: 1.0000
Epoch 86/1000
90/90 [==============================] - 1s 6ms/step - loss: 6.0340e-09 - accuracy: 1.0000
Epoch 87/1000
90/90 [==============================] - 1s 6ms/step - loss: 5.7479e-09 - accuracy: 1.0000
Epoch 88/1000
90/90 [==============

90/90 [==============================] - 1s 6ms/step - loss: 2.2842e-10 - accuracy: 1.0000
Epoch 156/1000
90/90 [==============================] - 1s 6ms/step - loss: 2.1841e-10 - accuracy: 1.0000
Epoch 157/1000
90/90 [==============================] - 1s 6ms/step - loss: 2.0881e-10 - accuracy: 1.0000
Epoch 158/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.9964e-10 - accuracy: 1.0000
Epoch 159/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.9097e-10 - accuracy: 1.0000
Epoch 160/1000
90/90 [==============================] - 1s 7ms/step - loss: 1.8261e-10 - accuracy: 1.0000
Epoch 161/1000
90/90 [==============================] - 1s 7ms/step - loss: 1.7463e-10 - accuracy: 1.0000
Epoch 162/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.6704e-10 - accuracy: 1.0000
Epoch 163/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.5980e-10 - accuracy: 1.0000
Epoch 164/1000
90/90 [==============================] - 1s 7m

90/90 [==============================] - 1s 6ms/step - loss: 1.0980e-11 - accuracy: 1.0000
Epoch 233/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.0628e-11 - accuracy: 1.0000
Epoch 234/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.0278e-11 - accuracy: 1.0000
Epoch 235/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.9547e-12 - accuracy: 1.0000
Epoch 236/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.6467e-12 - accuracy: 1.0000
Epoch 237/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.3474e-12 - accuracy: 1.0000
Epoch 238/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.0615e-12 - accuracy: 1.0000
Epoch 239/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.7837e-12 - accuracy: 1.0000
Epoch 240/1000
90/90 [==============================] - ETA: 0s - loss: 8.4516e-12 - accuracy: 1.00 - 1s 6ms/step - loss: 8.5176e-12 - accuracy: 1.0000
Epoch 241/1000


90/90 [==============================] - 1s 6ms/step - loss: 1.5247e-12 - accuracy: 1.0000
Epoch 309/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.4943e-12 - accuracy: 1.0000
Epoch 310/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.4645e-12 - accuracy: 1.0000
Epoch 311/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.4360e-12 - accuracy: 1.0000
Epoch 312/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.4080e-12 - accuracy: 1.0000
Epoch 313/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3808e-12 - accuracy: 1.0000
Epoch 314/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3534e-12 - accuracy: 1.0000
Epoch 315/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3252e-12 - accuracy: 1.0000
Epoch 316/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.2985e-12 - accuracy: 1.0000
Epoch 317/1000
90/90 [==============================] - 1s 6m

...

Epoch 616/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3910e-13 - accuracy: 1.0000
Epoch 617/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3872e-13 - accuracy: 1.0000
Epoch 618/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3834e-13 - accuracy: 1.0000
Epoch 619/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3797e-13 - accuracy: 1.0000
Epoch 620/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3759e-13 - accuracy: 1.0000
Epoch 621/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3722e-13 - accuracy: 1.0000
Epoch 622/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3683e-13 - accuracy: 1.0000
Epoch 623/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3646e-13 - accuracy: 1.0000
Epoch 624/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.3609e-13 - accuracy: 1.0000
Epoch 625/1000
90/90 [========================

90/90 [==============================] - 1s 6ms/step - loss: 1.1493e-13 - accuracy: 1.0000
Epoch 694/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.1466e-13 - accuracy: 1.0000
Epoch 695/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.1441e-13 - accuracy: 1.0000
Epoch 696/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.1417e-13 - accuracy: 1.0000
Epoch 697/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.1392e-13 - accuracy: 1.0000
Epoch 698/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.1368e-13 - accuracy: 1.0000
Epoch 699/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.1342e-13 - accuracy: 1.0000
Epoch 700/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.1318e-13 - accuracy: 1.0000
Epoch 701/1000
90/90 [==============================] - 1s 6ms/step - loss: 1.1294e-13 - accuracy: 1.0000
Epoch 702/1000
90/90 [==============================] - 1s 6m

90/90 [==============================] - 1s 6ms/step - loss: 9.8498e-14 - accuracy: 1.0000
Epoch 771/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.8320e-14 - accuracy: 1.0000
Epoch 772/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.8147e-14 - accuracy: 1.0000
Epoch 773/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.7969e-14 - accuracy: 1.0000
Epoch 774/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.7796e-14 - accuracy: 1.0000
Epoch 775/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.7628e-14 - accuracy: 1.0000
Epoch 776/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.7452e-14 - accuracy: 1.0000
Epoch 777/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.7279e-14 - accuracy: 1.0000
Epoch 778/1000
90/90 [==============================] - 1s 6ms/step - loss: 9.7107e-14 - accuracy: 1.0000
Epoch 779/1000
90/90 [==============================] - 1s 6m

90/90 [==============================] - 1s 6ms/step - loss: 8.6756e-14 - accuracy: 1.0000
Epoch 848/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.6624e-14 - accuracy: 1.0000
Epoch 849/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.6497e-14 - accuracy: 1.0000
Epoch 850/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.6365e-14 - accuracy: 1.0000
Epoch 851/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.6235e-14 - accuracy: 1.0000
Epoch 852/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.6107e-14 - accuracy: 1.0000
Epoch 853/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.5977e-14 - accuracy: 1.0000
Epoch 854/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.5850e-14 - accuracy: 1.0000
Epoch 855/1000
90/90 [==============================] - 1s 6ms/step - loss: 8.5725e-14 - accuracy: 1.0000
Epoch 856/1000
90/90 [==============================] - 1s 6m

90/90 [==============================] - 1s 6ms/step - loss: 7.7839e-14 - accuracy: 1.0000
Epoch 925/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.7739e-14 - accuracy: 1.0000
Epoch 926/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.7630e-14 - accuracy: 1.0000
Epoch 927/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.7523e-14 - accuracy: 1.0000
Epoch 928/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.7424e-14 - accuracy: 1.0000
Epoch 929/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.7321e-14 - accuracy: 1.0000
Epoch 930/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.7221e-14 - accuracy: 1.0000
Epoch 931/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.7123e-14 - accuracy: 1.0000
Epoch 932/1000
90/90 [==============================] - 1s 6ms/step - loss: 7.7026e-14 - accuracy: 1.0000
Epoch 933/1000
90/90 [==============================] - 1s 6m

In [13]:
train_dataset_features = train.copy()
train_dataset_labels = train.copy().pop('label')

tf.math.confusion_matrix(train_dataset_labels, model.predict(train_dataset_features))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2445,    0],
       [   0,  408]], dtype=int32)>

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

preds = np.array(model.predict(train_dataset_features)).astype(int)

accuracy = accuracy_score(train_dataset_labels.to_numpy(), preds)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(train_dataset_labels.to_numpy(), preds)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(train_dataset_labels.to_numpy(), preds)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(train_dataset_labels.to_numpy(), preds)
print('F1 score: %f' % f1)

Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1 score: 1.000000


In [15]:
test_dataset_features = test.copy()
test_dataset_labels = test.copy().pop('label')

tf.math.confusion_matrix(test_dataset_labels, model.predict(test_dataset_features))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[2390,    1],
       [   3,  460]], dtype=int32)>

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

preds = np.array(model.predict(test_dataset_features)).astype(int)

accuracy = accuracy_score(test_dataset_labels.to_numpy(), preds)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_dataset_labels.to_numpy(), preds)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_dataset_labels.to_numpy(), preds)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test_dataset_labels.to_numpy(), preds)
print('F1 score: %f' % f1)

Accuracy: 0.998598
Precision: 0.997831
Recall: 0.993521
F1 score: 0.995671
